In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
headers = {
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.94 Safari/537.36',
}
url = 'http://tradeinfo.sinyi.com.tw/metro/stationDetail.html?m=16'
res = requests.get(url,headers=headers)
soup = BeautifulSoup(res.text,'html.parser')
soup.select('.infoNew')[0].select('table')[0].select('tr')[3].select('td')[4]

<td class="fontWrap"><span class="ftcolor1">61.2萬</span><br/>
<span>(扣除車位價及車坪)</span>
</td>

In [2]:
res = requests.get('http://tradeinfo.sinyi.com.tw/metro/json/2016.json')
st_list = []
price_list = []
for data_dict in res.json():
    st_list.append(data_dict['station'])
    price_list.append(data_dict['price'])
house_price = pd.DataFrame({'station':st_list,'price':price_list})[['station','price']]
house_price.to_csv('C:/users/chuboy/desktop/tpe_house.csv',index=0)

In [38]:
from tabula import read_pdf
mrt_df = pd.DataFrame(columns=['date','weekday','vol'])
url_list = [
'https://www.tymetro.com.tw/upload/cht/attachment/16a6106a23720a7282c965a45e4c31c9.pdf',
'https://www.tymetro.com.tw/upload/cht/attachment/00a72886639a23819c4611466048ab85.pdf',
'https://www.tymetro.com.tw/upload/cht/attachment/8b6c650033e3b4e8ff92904c2a9732d6.pdf',
'https://www.tymetro.com.tw/upload/cht/attachment/a26efaee6d928eac65d60aecf1e78621.pdf',
'https://www.tymetro.com.tw/upload/cht/attachment/040bf7382548b7315a15770fe08cfa09.pdf',
'https://www.tymetro.com.tw/upload/cht/attachment/97c3c158ee4372c5935fa1295b5db62c.pdf',
'https://www.tymetro.com.tw/upload/cht/attachment/8684dee179fdb60945a9cb0021ed6f2d.pdf'
]
for url in url_list:
    pdf = read_pdf(url,encoding = 'big5', spreadsheet = True, pages = '1')
    df = pdf[:-2]
    df.columns = ['date','weekday','vol']
    df['year'] = '2017'
    df['month'] = df.date.str.extract(r'(\d)月')
    df['day'] = df.date.str.extract(r'(\d+)日')
    df.date = df.year+df.month+df.day
    df.date = pd.to_datetime(df.date,format='%Y%m%d')
    df.weekday = df.date.apply(lambda x:x.weekday()+1)
    df = df[['date','weekday','vol']]
    mrt_df = mrt_df.append(df)
mrt_df.to_csv('C:/users/chuboy/desktop/MRT/tyu_metro.csv',index=0)    

ImportError: cannot import name 'read_pdf'

In [247]:
import ezodf
def read_ods(filename, sheet_no=0, header=0):
    tab = ezodf.opendoc(filename=filename).sheets[sheet_no]
    return pd.DataFrame({col[header].value:[x.value for x in col[header+1:]]
                         for col in tab.columns()})
mrt_df = pd.DataFrame(columns=['station','date','weekday','vol'])
for month in range(3,10):
    df = read_ods('C:/users/chuboy/desktop/MRT/20170'+str(month)+'_cht.ods')
    df = df.dropna(how='all').T.dropna(how='all')
    df = df[1:]
    df['st'] = df.index
    df = pd.melt(df,id_vars=['st'])
    
    df.columns = ['station','day','vol']
    df.day = df.day.apply(lambda x:x+1).astype(str)
    df['month'] = str(month)
    df['year'] = '2017'
    df['date'] = df.year+df.month+df.day
    df.date = pd.to_datetime(df.date,format='%Y%m%d')
    df['weekday'] = df.date.apply(lambda x:x.weekday()+1)
    df = df[['station','date','weekday','vol']]
    mrt_df = mrt_df.append(df)
#mrt_df.to_csv('C:/users/chuboy/desktop/MRT/tpe_metro.csv',index=0)

In [51]:
airport = pd.DataFrame(columns=['date','in','out','inout','outin','intr','outtr'])
for month in range(3,10):
    ap = pd.read_html('C:/users/chuboy/desktop/MRT/20170'+str(month)+'.htm')[0].iloc[1:-1,[0,1,4,7,10,13,16]]
    #ap.columns = ['date','入境','出境','入境過境','出境過境','入境轉機','出境轉機']
    ap.columns = ['date','in','out','inout','outin','intr','outtr']
    ap.date = pd.to_datetime(ap.date,format='%Y%m%d')
    airport = airport.append(ap)
airport.to_csv('C:/users/chuboy/desktop/MRT/airport.csv',index=0)    
airport.head()

,date,in,out,inout,outin,intr,outtr
1,2017-03-01,61154,43285,533,849,5618,5579
2,2017-03-02,55779,41330,1273,1029,6779,6975
3,2017-03-03,54756,41111,1036,1170,6775,7173
4,2017-03-04,52798,42399,898,1120,6424,6697
5,2017-03-05,57764,51822,724,764,5631,6782


In [ ]:
import pandas as pd
url_list = [
'http://web.metro.taipei/RidershipCounts/c/8504.htm',
'http://web.metro.taipei/RidershipCounts/c/8505.htm',
'http://web.metro.taipei/RidershipCounts/c/8505.htm',
'http://web.metro.taipei/RidershipCounts/c/8506.htm',
'http://web.metro.taipei/RidershipCounts/c/8507.htm',
'http://web.metro.taipei/RidershipCounts/c/8508.htm',
'http://web.metro.taipei/RidershipCounts/c/8509.htm',
'http://web.metro.taipei/RidershipCounts/c/8510.htm',
'http://web.metro.taipei/RidershipCounts/c/8511.htm',
'http://web.metro.taipei/RidershipCounts/c/8512.htm',
'http://web.metro.taipei/RidershipCounts/c/8601.htm',
'http://web.metro.taipei/RidershipCounts/c/8602.htm',
'http://web.metro.taipei/RidershipCounts/c/8603.htm',
]
#木柵線通車
df = pd.DataFrame(columns=['營運日','星期','總運量'])
for u in url_list:
    temp = pd.read_html(u)[0].iloc[2:,].dropna()
    df = df.append(temp)
df = df.T.dropna().T
df.columns=['營運日','星期','總運量']
df.to_csv('C:/users/chuboy/desktop/木柵線通車.csv',index=0,encoding='big5')
df.head()